# testing

## testing

### testing_trace

In [ ]:
union testing_trace =
    | Console
    | Trace
    | TraceRaw
    | Silent

### __expect

In [ ]:
inl rec __expect fn trace' name b a =
    inl result = fn a b
    inl result =
        result || join result
    inl get_raw_text () =
        backend_switch {
            Fsharp = fun () => $'$"{!name} / actual: %A{!a} / expected: %A{!b}"' : string
            Python = fun () => $'f"{!name} / actual: {!a} / expected: {!b}"' : string
        }
    match trace' with
    | Console =>
        inl text = get_raw_text ()
        text |> console.write_line
        text
    | Trace =>
        trace Info (fun () => name) fun () => { actual = a; expected = b }
        get_raw_text ()
    | TraceRaw =>
        inl text = get_raw_text ()
        trace_raw Info fun () => text
        text
    | Silent => reflection.nameof { __expect }
    |> assert result

### __assert_approx_eq

In [ ]:
inl rec __assert_approx_eq trace e b a =
    __expect
        (fun a b => abs (b - a) < (e |> optionm.defaultWith 0.00000001))
        trace
        (reflection.nameof { __assert_approx_eq })
        b
        a

inl _assert_approx_eq e b a =
    __assert_approx_eq Console e b a

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

12.345f64
|> _assert_approx_eq (Some 0.0001f64) 12.345f64

.py output (Cuda):
__assert_approx_eq / actual: 12.345 / expected: 12.345

.rs output:
__assert_approx_eq / actual: 12.345 / expected: 12.345

.ts output:
__assert_approx_eq / actual: 12.345 / expected: 12.345

.py output:
__assert_approx_eq / actual: 12.345 / expected: 12.345



.fsx output:
__assert_approx_eq / actual: 12.345 / expected: 12.345


In [ ]:
//// test
//// print_code

1f64
|> __assert_approx_eq Console (Some 3) 2

let rec closure0 (v0 : string) () : unit =
    let v1 : (string -> unit) = System.Console.WriteLine
    v1 v0
and method0 () : unit =
    let v0 : string = "__assert_approx_eq"
    let v1 : string = $"{v0} / actual: %A{1.0} / expected: %A{2.0}"
    let v4 : unit = ()
    let v5 : (unit -> unit) = closure0(v1)
    let v6 : unit = (fun () -> v5 (); v4) ()
    ()
method0()

__assert_approx_eq / actual: 1.0 / expected: 2.0


In [ ]:
//// test
//// print_code

(dyn 1f64)
|> _assert_approx_eq (Some 3) 2

let rec method1 (v0 : bool) : bool =
    v0
and closure0 (v0 : string) () : unit =
    let v1 : (string -> unit) = System.Console.WriteLine
    v1 v0
and method0 () : unit =
    let v0 : float = 1.0
    let v1 : float = 2.0 - v0
    let v2 : float =  -v1
    let v3 : bool = v1 >= v2
    let v4 : float =
        if v3 then
            v1
        else
            v2
    let v5 : bool = v4 < 3.0
    let v7 : bool =
        if v5 then
            true
        else
            method1(v5)
    let v8 : string = "__assert_approx_eq"
    let v9 : string = $"{v8} / actual: %A{v0} / expected: %A{2.0}"
    let v12 : unit = ()
    let v13 : (unit -> unit) = closure0(v9)
    let v14 : unit = (fun () -> v13 (); v12) ()
    let v16 : bool = v7 = false
    if v16 then
        failwith<unit> v9
method0()

__assert_approx_eq / actual: 1.0 / expected: 2.0


### __assert_eq

In [ ]:
inl rec __assert_eq trace b a =
    __expect (=) trace (reflection.nameof { __assert_eq }) b a

inl _assert_eq b a =
    __assert_eq Console b a

### __assert_eq'

In [ ]:
inl rec __assert_eq' trace b a =
    __expect (=.) trace (reflection.nameof { __assert_eq' }) b a

inl _assert_eq' b a =
    __assert_eq' Console b a

### __assert_ne

In [ ]:
inl rec __assert_ne trace b a =
    __expect (<>.) trace (reflection.nameof { __assert_ne }) b a

inl _assert_ne b a =
    __assert_ne Console b a

### __assert_gt

In [ ]:
inl rec __assert_gt trace b a =
    __expect (>) trace (reflection.nameof { __assert_gt }) b a

inl _assert_gt b a =
    __assert_gt Console b a

### __assert_ge

In [ ]:
inl rec __assert_ge trace b a =
    __expect (>=) trace (reflection.nameof { __assert_ge }) b a

inl _assert_ge b a =
    __assert_ge Console b a

### __assert_lt

In [ ]:
inl rec __assert_lt trace b a =
    __expect (<) trace (reflection.nameof { __assert_lt }) b a

inl _assert_lt b a =
    __assert_lt Console b a

### __assert_le

In [ ]:
inl rec __assert_le trace b a =
    __expect (<=) trace (reflection.nameof { __assert_le }) b a

inl _assert_le b a =
    __assert_le Console b a

### __assert

In [ ]:
inl rec __assert fn trace b a =
    __expect fn trace (reflection.nameof { __assert }) a b

inl _assert fn b a =
    __assert fn Console b a

### __assert_between

In [ ]:
inl rec __assert_between trace a b actual =
    inl assert_between actual (a, b) =
        __assert_ge Silent a actual
        __assert_le Silent b actual
        true
    __expect assert_between trace (reflection.nameof { __assert_between }) (a, b) actual

inl _assert_between a b actual =
    __assert_between Console a b actual

### _assert_fn

In [ ]:
inl rec _assert_fn fn list =
    list
    |> listm.rev
    |> listm.map fun input, expected => join
        input
        |> fn
        |> resultm.get
        |> fun x =>
            inl expected' = join expected
            inl name = reflection.nameof { _assert_fn }
            try
                fun () =>
                    console.write_line ""
                    trace Verbose
                        fun () => name
                        fun () => { input }
                    x
                    |> sm'.format
                    |> _assert_eq' (expected' |> sm'.format)
                    true
                fun ex =>
                    trace Critical
                        fun () =>
                            backend_switch {
                                Fsharp = fun () => $'$"{!name} / error"' : string
                                Python = fun () => $'f"{!name} / error"' : string
                            }
                        fun () => { ex expected }
                    Some false
            |> optionm.value
    |> listm'.filter not
    |> function
        | [] => ()
        | x => x |> sm'.format_debug |> failwith

## fsharp

### __assert_contains

In [ ]:
inl rec __assert_contains forall t u. (trace : testing_trace) (b : t) (a : u) : () =
    __expect
        fun a b =>
            a
            |> $'List.ofSeq'
            |> fun x => x : listm'.list' t
            |> $'List.tryFind' ((=) b)
            |> optionm'.unbox
            |> fun (x : option t) => x <> None
        trace
        // TODO: forall nameof (Cannot dyn a forall into a runtime var.)
        // Metavars that are not part of the enclosing function's signature are not allowed. They need to be values.
        // Got: {__assert_contains : testing_trace -> _ -> _ -> ()} -> string
        // (reflection.nameof { __assert_contains })
        "__assert_contains"
        b
        a

inl _assert_contains b a =
    __assert_contains Console b a

In [ ]:
//// test

;[ "a"; "b"; "c" ]
|> _assert_contains "b"

__assert_contains / actual: [|"a"; "b"; "c"|] / expected: "b"


In [ ]:
//// test

"abcd"
|> _assert_contains 'b'

__assert_contains / actual: "abcd" / expected: 'b'


### _throws

In [ ]:
inl _throws (fn : () -> ()) : option exn =
    inl none = None : option exn
    inl some (s : exn) = Some s
    backend_switch {
        Fsharp = fun () =>
            $'try !fn (); !none with ex -> ex |> !some ' : option exn
        Python = fun () =>
            $'fn = !fn '
            $'none = !none '
            $'some = !some '
            $'try: fn(); x = none '
            $'except Exception as ex: x = some(ex)'
            $'x' : option exn
    }

### print_and_return

In [ ]:
inl rec print_and_return x =
    inl name = reflection.nameof { print_and_return }
    backend_switch {
        Fsharp = fun () => $'printfn $"{!name} / x: {!x}"' : ()
        Python = fun () => $'print(f"{!name} / x: {!x}")' : ()
    }
    x